In [ ]:
#pip install xgboost

In [53]:
# 경로 설정 
import os
os.chdir('C:\\Users\\billk\\weathercontest\\data')

In [54]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


from xgboost import plot_importance
from xgboost import XGBClassifier

In [55]:
def metrics(y_test, pred):
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_score = roc_auc_score(y_test, pred)
    print('정확도 : {0:.2f}, 정밀도 : {1:.2f}, 재현율 : {2:.2f}'.format(accuracy, precision, recall))
    print('f1-score : {0:.2f}, auc : {1:.2f}'.format(f1, roc_score))

# 데이터 처리 및 탐색

In [57]:
raw_df = pd.read_csv("merge_final_real.csv", encoding='utf-8-sig')

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [15]:
raw_df['1day_yn'] = 0.0
raw_df['2day_yn'] = 0.0

In [16]:
# 열을 밀어서 출력, 맨 첫날, 마지막 날의 경우 산사태가 발생하지 않음.
raw_df['1day_yn'][1:-1] = raw_df['landslide'][0:-2]
raw_df['2day_yn'][2:-1] = raw_df['landslide'][0:-3]

<ipython-input-16-b6da74698f00>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['1day_yn'][1:-1] = raw_df['landslide'][0:-2]
<ipython-input-16-b6da74698f00>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['2day_yn'][2:-1] = raw_df['landslide'][0:-3]


In [36]:
select_df = raw_df[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn', 'hr1MaxRn', 'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLANT_TYP','SLDPT_TPCD','PRDN_FOM_C','SLANT_TYP','SLDPT_TPCD','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD','FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT','1day_yn']]
cleaned_df_1 = select_df.fillna(0)
cleaned_df_1.info()
cleaned_df_1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2104844 entries, 0 to 2104843
Data columns (total 32 columns):
 #   Column      Dtype  
---  ------      -----  
 0   PRRCK_LARG  float64
 1   PRRCK_MDDL  float64
 2   LOCTN_ALTT  float64
 3   LOCTN_GRDN  float64
 4   CLZN_CD     float64
 5   TPGRP_TPCD  float64
 6   PRDN_FOM_C  float64
 7   SLANT_TYP   float64
 8   SLDPT_TPCD  float64
 9   SCSTX_CD    float64
 10  SLTP_CD     float64
 11  stnId       int64  
 12  avgTa       float64
 13  minTa       float64
 14  STORUNST    float64
 15  FROR_CD     float64
 16  FRTP_CD     float64
 17  KOFTR_GROU  float64
 18  DMCLS_CD    float64
 19  AGCLS_CD    float64
 20  HEIGHT      float64
 21  mi10MaxRn   float64
 22  hr1MaxRn    float64
 23  sumRnDur    float64
 24  sumRn       float64
 25  maxInsWs    float64
 26  maxWs       float64
 27  avgWs       float64
 28  minRhm      float64
 29  avgRhm      float64
 30  n99Rn       float64
 31  1day_yn     float64
dtypes: float64(31), int64(1)
memory 

,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,...,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,1day_yn
0,2.0,24.0,306.1,18.7,3.0,12.0,1.0,1.0,20.0,2.0,...,0.0,0.0,0.0,9.3,6.0,1.8,25.0,48.9,0.0,0.0
1,2.0,24.0,306.1,18.7,3.0,12.0,1.0,1.0,20.0,2.0,...,0.0,0.0,0.0,6.4,4.9,1.0,31.0,67.0,0.0,0.0
2,2.0,24.0,306.1,18.7,3.0,12.0,1.0,1.0,20.0,2.0,...,0.0,0.0,0.4,5.7,3.7,0.8,36.0,77.8,0.0,0.0
3,2.0,24.0,306.1,18.7,3.0,12.0,1.0,1.0,20.0,2.0,...,0.0,0.0,0.5,5.8,2.8,1.0,35.0,73.9,0.0,0.0
4,2.0,24.0,306.1,18.7,3.0,12.0,1.0,1.0,20.0,2.0,...,0.0,0.0,0.0,8.3,4.5,1.4,33.0,60.3,0.0,0.0


## 1일 후 예측

In [37]:
y = cleaned_df_1['1day_yn']
X = cleaned_df_1.drop('1day_yn', axis =1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 11)
print(X_train.shape, X_test.shape)

(1683875, 31) (420969, 31)


In [20]:
xgb = XGBClassifier(n_estimators=500, learning_rate = 0.1, max_depth = 4)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

C:\Users\billk\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:12:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [21]:
metrics(y_test, xgb_pred)

정확도 : 1.00, 정밀도 : 0.91, 재현율 : 0.98
f1-score : 0.94, auc : 0.99


### 하이퍼 파라미터 튜닝 후 예측

In [ ]:
xgb = XGBClassifier()

xgb_param_grid = {
    'n_estimators' : [100, 200, 400, 600],
    'learning_rate' : [0.01, 0.05, 0.1, 0.15, 0.2],
    'max_depth' : [4, 6, 8, 10, 12],
}
# sklearn.metrics.SCORERS.keys()
xgb_grid = GridSearchCV(xgb, param_grid = xgb_param_grid, scoring="roc_auc", n_jobs= 1, verbose = 1)
xgb_grid.fit(X_train, y_train)

In [ ]:
print("최고 평균 auc : {0:.4f}".format(xgb_grid.best_score_))
print("최고의 파라미터 : ", xgb_grid.best_params_)

In [ ]:
result = pd.DataFrame(xgb_grid.cv_results_)
result.sort_values(by=['rank_test_score'], inplace=True)

In [ ]:
result[['params', 'mean_test_score', 'rank_test_score']].head(10)

## 최고의 파라미터 값으로 예측 실시

In [ ]:
xgb = XGBClassifier(n_estimators=400, learning_rate = 0.1, max_depth = 3)
evals = [(X_test, y_test)]
xgb.fit(X_train, y_train, early_stopping_rounds = 100, eval_metric = "logloss", eval_set = evals, verbose = 1)

## 2일 후 예측

In [38]:
#분석을 위해 음수 값을 가지는 칼럼들을 제외하고 결측치를 0으로 대체한다.
select_df = raw_df[['PRRCK_LARG','PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLANT_TYP','SLDPT_TPCD','SCSTX_CD','SLTP_CD','stnId','avgTa', 'minTa','STORUNST','FROR_CD','FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT','mi10MaxRn','hr1MaxRn', 'sumRnDur', 'sumRn', 
       'maxInsWs', 'maxWs', 'avgWs', 'minRhm','avgRhm', 'n99Rn','2day_yn']]
cleaned_df_2 = select_df.fillna(0)
cleaned_df_2.info()
cleaned_df_2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2104844 entries, 0 to 2104843
Data columns (total 32 columns):
 #   Column      Dtype  
---  ------      -----  
 0   PRRCK_LARG  float64
 1   PRRCK_MDDL  float64
 2   LOCTN_ALTT  float64
 3   LOCTN_GRDN  float64
 4   CLZN_CD     float64
 5   TPGRP_TPCD  float64
 6   PRDN_FOM_C  float64
 7   SLANT_TYP   float64
 8   SLDPT_TPCD  float64
 9   SCSTX_CD    float64
 10  SLTP_CD     float64
 11  stnId       int64  
 12  avgTa       float64
 13  minTa       float64
 14  STORUNST    float64
 15  FROR_CD     float64
 16  FRTP_CD     float64
 17  KOFTR_GROU  float64
 18  DMCLS_CD    float64
 19  AGCLS_CD    float64
 20  HEIGHT      float64
 21  mi10MaxRn   float64
 22  hr1MaxRn    float64
 23  sumRnDur    float64
 24  sumRn       float64
 25  maxInsWs    float64
 26  maxWs       float64
 27  avgWs       float64
 28  minRhm      float64
 29  avgRhm      float64
 30  n99Rn       float64
 31  2day_yn     float64
dtypes: float64(31), int64(1)
memory 

,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,...,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,2day_yn
0,2.0,24.0,306.1,18.7,3.0,12.0,1.0,1.0,20.0,2.0,...,0.0,0.0,0.0,9.3,6.0,1.8,25.0,48.9,0.0,0.0
1,2.0,24.0,306.1,18.7,3.0,12.0,1.0,1.0,20.0,2.0,...,0.0,0.0,0.0,6.4,4.9,1.0,31.0,67.0,0.0,0.0
2,2.0,24.0,306.1,18.7,3.0,12.0,1.0,1.0,20.0,2.0,...,0.0,0.0,0.4,5.7,3.7,0.8,36.0,77.8,0.0,0.0
3,2.0,24.0,306.1,18.7,3.0,12.0,1.0,1.0,20.0,2.0,...,0.0,0.0,0.5,5.8,2.8,1.0,35.0,73.9,0.0,0.0
4,2.0,24.0,306.1,18.7,3.0,12.0,1.0,1.0,20.0,2.0,...,0.0,0.0,0.0,8.3,4.5,1.4,33.0,60.3,0.0,0.0


In [39]:
y = cleaned_df_2['2day_yn']
X = cleaned_df_2.drop('2day_yn', axis =1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 11)
print(X_train.shape, X_test.shape)

(1683875, 31) (420969, 31)


In [40]:
xgb = XGBClassifier(n_estimators=500, learning_rate = 0.1, max_depth = 4)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

C:\Users\billk\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:53:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [42]:
metrics(y_test, xgb_pred)

정확도 : 1.00, 정밀도 : 0.89, 재현율 : 0.93
f1-score : 0.91, auc : 0.97


# LightGBM

In [41]:
from lightgbm import LGBMClassifier, plot_importance

In [ ]:
lgb = LGBMClassifier(n_estimaotrs = 400)
lgb.fit(X_train, y_train)

In [ ]:
lgb_pred = lgb.predict(X_test)
metrics(y_test, lgb_pred)